<a href="https://colab.research.google.com/github/a3caicedo/TechnicalTest/blob/main/TechnicalTestSecurityAutomationEngineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install OTXv2

In [6]:
#https://github.com/AlienVault-OTX/OTX-Python-SDK/blob/master/examples/is_malicious/get_malicious.py

#!/usr/bin/env python

import IndicatorTypes

# Get a nested key from a dict, without having to do loads of ifs
def getValue(results, keys):
    if type(keys) is list and len(keys) > 0:

        if type(results) is dict:
            key = keys.pop(0)
            if key in results:
                return getValue(results[key], keys)
            else:
                return None
        else:
            if type(results) is list and len(results) > 0:
                return getValue(results[0], keys)
            else:
                return results
    else:
        return results

def query_ip(otx, ip):
    alerts = []
    result = otx.get_indicator_details_by_section(IndicatorTypes.IPv4, ip, 'general')

    # Return nothing if it's in the whitelist
    validation = getValue(result, ['validation'])
    if not validation:
        pulses = getValue(result, ['pulse_info', 'pulses'])
        if pulses:
            for pulse in pulses:
                if 'name' in pulse:
                    alerts.append(pulse['name'])

    return alerts

In [7]:
from numpy import nan
from OTXv2 import OTXv2
from OTXv2 import IndicatorTypes
import pandas as pd
import datetime

otx = OTXv2("ApiKey")
ips = ['101.43.65.150', '101.43.32.173', '78.128.113.14', '106.55.153.204', '104.243.41.123']

pulses = pd.DataFrame()

for ip in ips:
  alerts = query_ip(otx, ip)
  for alert in alerts:
    values = [x.strip() for x in alert.split('|')]
    date = None if len(values) == 1 else datetime.datetime.strptime(values[1], '%m/%d/%Y')
    pulses = pulses.append({'ip': ip, 'inPulse': values[0], 'Date': date}, ignore_index=True)

pulses.to_csv('report.csv',  index=False, encoding='utf-8')